In [1]:
from sklearn import metrics, datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import numpy as np
%matplotlib inline

## 回歸問題
常見的評估指標有
- MAE
- MSE
- R-square

我們隨機生成(X, y)資料，然後使用線性回歸模型做預測，再使用 MAE, MSE, R-square 評估

------

## sklearn.datasets.make_regression

```
sklearn.datasets.make_regression(n_samples=100, n_features=100, n_informative=10, n_targets=1, bias=0.0, effective_rank=None, tail_strength=0.5, noise=0.0, shuffle=True, coef=False, random_state=None)
```

Generate a random regression problem.

The input set can either be well conditioned (by default) or have a low rank-fat tail singular profile. See make_low_rank_matrix for more details.

The output is generated by applying a (potentially biased) random linear regression model with n_informative nonzero regressors to the previously generated input and some gaussian centered noise with some adjustable scale

### Parameters:	
#### n_samples : int, optional (default=100)
The number of samples.

#### n_features : int, optional (default=100)
The number of features.

#### n_informative : int, optional (default=10)
The number of informative features, i.e., the number of features used to build the linear model used to generate the output.

#### n_targets : int, optional (default=1)
The number of regression targets, i.e., the dimension of the y output vector associated with a sample. By default, the output is a scalar.

#### bias : float, optional (default=0.0)
The bias term in the underlying linear model.

#### effective_rank : int or None, optional (default=None)
if not None:
The approximate number of singular vectors required to explain most of the input data by linear combinations. Using this kind of singular spectrum in the input allows the generator to reproduce the correlations often observed in practice.

if None:
The input set is well conditioned, centered and gaussian with unit variance.

#### tail_strength : float between 0.0 and 1.0, optional (default=0.5)
The relative importance of the fat noisy tail of the singular values profile if effective_rank is not None.

#### noise : float, optional (default=0.0)
The standard deviation of the gaussian noise applied to the output.

#### shuffle : boolean, optional (default=True)
Shuffle the samples and the features.

#### coef : boolean, optional (default=False)
If True, the coefficients of the underlying linear model are returned.

#### random_state : int, RandomState instance or None (default)
Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls. See Glossary.

### Returns:	
#### X : array of shape [n_samples, n_features]
The input samples.

#### y : array of shape [n_samples] or [n_samples, n_targets]
The output values.

#### coef : array of shape [n_features] or [n_features, n_targets], optional
The coefficient of the underlying linear model. It is returned only if coef is True.

------

In [2]:
X, y = datasets.make_regression(n_features=1, random_state=42, noise=4) # 生成資料
model = LinearRegression() # 建立回歸模型
model.fit(X, y) # 將資料放進模型訓練
prediction = model.predict(X) # 進行預測
mae = metrics.mean_absolute_error(prediction, y) # 使用 MAE 評估
mse = metrics.mean_squared_error(prediction, y) # 使用 MSE 評估
r2 = metrics.r2_score(prediction, y) # 使用 r-square 評估
print("MAE: ", mae)
print("MSE: ", mse)
print("R-square: ", r2)

MAE:  2.841797252565566
MSE:  12.48868006739824
R-square:  0.9916581036260311


## 分類問題
常見的評估指標有
- AUC
- F1-Score (Precision, Recall)

In [3]:
cancer = datasets.load_breast_cancer() # 我們使用 sklearn 內含的乳癌資料集
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=50, random_state=0)

In [6]:
print(f'type(cancer.data) = {type(cancer.data)}, type(cancer.target) = {type(cancer.target)}')


type(cancer.data) = <class 'numpy.ndarray'>, type(cancer.target) = <class 'numpy.ndarray'>


In [8]:
print(f'cancer.data.shape = {cancer.data.shape}, cancer.target.shape = {cancer.target.shape}')

cancer.data.shape = (569, 30), cancer.target.shape = (569,)


In [9]:
print(y_test) # 測試集中的 label

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0]


In [10]:
y_pred = np.random.random((50,)) # 我們先隨機生成 50 筆預測值，範圍都在 0~1 之間，代表機率值

In [11]:
print(y_pred)

[0.66123034 0.73586381 0.72938425 0.529258   0.75626125 0.82384875
 0.68325569 0.18261265 0.50293228 0.03819507 0.30233347 0.4345511
 0.44762763 0.35988371 0.73888824 0.67688205 0.42393111 0.61809024
 0.6509393  0.09195089 0.4373601  0.44862276 0.99450543 0.72775914
 0.30850245 0.93057208 0.76880842 0.69396679 0.75876336 0.04299818
 0.27339911 0.50209606 0.86921257 0.95531364 0.082625   0.58871981
 0.08126352 0.48662877 0.06670626 0.75542061 0.37841451 0.00874121
 0.45649367 0.10175114 0.63108744 0.66567501 0.22915074 0.06882581
 0.38120775 0.31087305]


### AUC

In [12]:
auc = metrics.roc_auc_score(y_test, y_pred) # 使用 roc_auc_score 來評估。 **這邊特別注意 y_pred 必須要放機率值進去!**
print("AUC: ", auc) # 得到結果約 0.5，與亂猜的結果相近，因為我們的預測值是用隨機生成的

AUC:  0.4787775891341256


## F1-Score

In [13]:
threshold = 0.5 
y_pred_binarized = np.where(y_pred>threshold, 1, 0) # 使用 np.where 函數, 將 y_pred > 0.5 的值變為 1，小於 0.5 的為 0

In [15]:
y_pred_binarized

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0])

In [16]:
# 都使用 metrics 套件
f1 = metrics.f1_score(y_test, y_pred_binarized) # 使用 F1-Score 評估
precision = metrics.precision_score(y_test, y_pred_binarized) # 使用 Precision 評估
recall  = metrics.recall_score(y_test, y_pred_binarized) # 使用 recall 評估
print("F1-Score: ", f1) 
print("Precision: ", precision)
print("Recall: ", recall)

F1-Score:  0.5357142857142857
Precision:  0.6
Recall:  0.4838709677419355


## 作業
請參考 F1-score 的公式與[原始碼](https://github.com/scikit-learn/scikit-learn/blob/bac89c2/sklearn/metrics/classification.py#L620)，試著寫出 F2-Score 的計算函數